<a href="https://colab.research.google.com/github/johannnamr/Discrepancy-based-inference-using-QMC/blob/main/Helper-functions/Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utils

Notebook containing all necessary functions for the conducted analyses

### Imports

In [ ]:
! pip install --upgrade pip # update pip to latest version
! pip install qmcpy --quiet
! pip install pytictoc --quiet
! pip install POT --quiet

     |████████████████████████████████| 1.5MB 6.8MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
import numpy as np
import scipy.stats as stats # used for inverse Gaussian
import scipy.spatial.distance as distance # distance used for kernel
from sklearn.gaussian_process.kernels import Matern # Matern kernel
import qmcpy # QMC points
from pytictoc import TicToc # timer
import ot # Wasserstein distance and Sinkhorn divergence

In [ ]:
# autodiff
import jax.numpy as jnp
from jax import jacfwd
from jax import ops
from jax import lax
from copy import deepcopy

## Inverse Gaussian

Function generating standard normals using the inverse of the univariate Gaussian CDF: 

In [ ]:
def normals_inv(u):
  # avoid origin
  u[u==0] = np.nextafter(0, 1)
  # create standard normal samples
  z = stats.norm.ppf(u, loc=0, scale=1)
  return z

## Box-Muller transformation

Box-Muller transformation:

In [ ]:
def boxmuller(unif1,unif2):
  u1 = np.sqrt(-2*np.log(unif1))*np.cos(2*np.pi*unif2)
  u2 = np.sqrt(-2*np.log(unif1))*np.sin(2*np.pi*unif2)
  return np.transpose(np.vstack([u1,u2]))

Function generating standard normals using the box-muller transformation:

In [ ]:
def normals(n, d, unif, sv=False):

    # avoid origin
    unif[unif==0] = np.nextafter(0, 1)

    # if d is odd, add one dimension
    if d % 2 != 0:
      dim = d + 1
    else:
      dim = d

    # expand dimensions for SV model
    if sv == True:
      dim = 2+2*d

    # create standard normal samples
    u = np.zeros((n,dim))
    for i in np.arange(0,dim,2):
      u[:,i:(i+2)] = boxmuller(unif[:,i],unif[:,(i+1)])

    # if d is odd, drop one dimension
    if d % 2 != 0 or sv == True:
      u = np.delete(u,-1,1)

    return u

## Kernel

Gaussian kernel $k(x,y)$, its gradient w.r.t. first element $\nabla_1k(x,y)$ and its second derivative w.r.t. to the second and first argument $\nabla_2\nabla_1k(x,y)$:

In [ ]:
def k(x,y,kernel,l,c,b,nu,grad=True,sparse=False): 

    if sparse == True:
      x = x.astype('float32')
      y = y.astype('float32')
    
    # dimensions
    d = x.shape[1]
    dims = np.arange(d)
    
    if kernel=='gaussian':
      # kernel
      r = distance.cdist(x,y,'sqeuclidean')
      K = np.exp(-(1/(2*l**2))*r)
      if grad:
        # first derivative
        K_extended = K[np.newaxis,:,:]
        diff_extended = (x[np.newaxis,:]-y[:,np.newaxis])
        diff_extended = np.swapaxes(diff_extended,0,2)
        grad_1 = -1*diff_extended*(1/l**2)*K_extended
        # second derivative
        diff_diffT = np.einsum('inm,jnm->ijnm',diff_extended,diff_extended)
        grad_21 = (1/l**2)*(np.eye(d)[:,:,np.newaxis,np.newaxis]-diff_diffT*(1/l**2))*K

    if kernel=='imq':
      # kernel
      r = distance.cdist(x,y,'sqeuclidean')
      K = (c**2+r)**b
      if grad:
        # first derivative
        r_extended = r[np.newaxis,:,:]
        diff_extended = (x[np.newaxis,:]-y[:,np.newaxis])
        diff_extended = np.swapaxes(diff_extended,0,2)
        grad_1 = 2.*diff_extended*b*(c**2+r_extended)**(b-1.)
        # second derivative
        diagonal = -4.*b*(b-1.)*(diff_extended**2)*((c**2+r_extended)**(b-2.))-2.*b*((c**2+r_extended)**(b-1.))
        diff_diffT = np.einsum('inm,jnm->ijnm',diff_extended,diff_extended)
        grad_21 = -4.*b*(b-1.)*diff_diffT*np.power(c**2+r_extended[np.newaxis:,:,:],b-2.)
        grad_21[range(d),range(d),:,:] = diagonal
    
    if kernel=='matern':
      km = Matern(length_scale=l,nu=nu)
      K = km(x,y)

    if grad:
      return list([K, grad_1, grad_21])
    else:
      return list([K])

## MMD$^2$

MMD$^2$ approximation:

In [ ]:
def MMD_approx(n,m,kxx,kxy,kyy,stat_type='v'):
  if stat_type=='u':
    # first sum
    np.fill_diagonal(kxx, np.repeat(0,n))
    sum1 = np.sum(kxx)
    
    # second sum
    sum2 = np.sum(kxy)
    
    # third sum
    np.fill_diagonal(kyy, np.repeat(0,m))
    sum3 = np.sum(kyy)
    
    return (1/(n*(n-1)))*sum1-(2/(m*n))*sum2+(1/(m*(m-1)))*sum3
    
  if stat_type=='v':
    # first sum
    np.fill_diagonal(kxx, np.repeat(0,n))
    sum1 = np.sum(kxx)
    
    # second sum
    sum2 = np.sum(kxy)
    
    # third sum
    np.fill_diagonal(kyy, np.repeat(0,m))
    sum3 = np.sum(kyy)
    
    return (1/n**2)*sum1-(2/(m*n))*sum2+(1/m**2)*sum3

MMD$^2$ gradient $\hat{J}$:

In [ ]:
def grad_MMD(p,n,m,grad_g,k1xx,k1xy,stat_type='v'):
    
    if stat_type == 'u':
      # first sum
      prod1 = np.squeeze(np.einsum('ilj,imjk->lmjk', grad_g, np.expand_dims(k1xx,axis=1)))
      if prod1.ndim==2:
        np.fill_diagonal(prod1[:,:], 0)
        sum1 = np.sum(prod1)
      else:
        for i in range(p):
          np.fill_diagonal(prod1[i,:,:], 0)
        sum1 = np.einsum('ijk->i',prod1)
    
      # second sum
      prod2 = np.squeeze(np.einsum('ilj,imjk->lmjk', grad_g, np.expand_dims(k1xy,axis=1)))
      if prod2.ndim==2:
        sum2 = np.sum(prod2)
      else:
        sum2 = np.einsum('ijk->i',prod2)
    
      return (2/(n*(n-1)))*sum1-(2/(n*m))*sum2

    if stat_type == 'v':
      # first sum
      prod1 = np.squeeze(np.einsum('ilj,imjk->lmjk', grad_g, np.expand_dims(k1xx,axis=1)))
      if prod1.ndim==2:
        np.fill_diagonal(prod1[:,:], 0)
        sum1 = np.sum(prod1)
      else:
        for i in range(p):
          np.fill_diagonal(prod1[i,:,:], 0)
        sum1 = np.einsum('ijk->i',prod1)
    
      # second sum
      prod2 = np.squeeze(np.einsum('ilj,imjk->lmjk', grad_g, np.expand_dims(k1xy,axis=1)))
      if prod2.ndim==2:
        sum2 = np.sum(prod2)
      else:
        sum2 = np.einsum('ijk->i',prod2)
    
      return (2/n**2)*sum1-(2/(n*m))*sum2

## Information metric

Approximation of the informatin metric $g_U(\theta)$:

In [ ]:
def g_approx(p,n,grad_g,k21xx):
    
    # sum
    grad_g_T = np.einsum('ijk -> jik',grad_g)
    prod1 = np.einsum('ijk, jlkm -> ilkm', grad_g_T, k21xx)
    prod2 = np.einsum('ijkl,jmk->imkl', prod1, grad_g)
    for i in range(p):
        np.fill_diagonal(prod2[i,i,:,:], 0)
    gsum = np.einsum('ijkl->ij', prod2)
    
    return 1/(n*(n-1))*gsum

## Generators

Generator $G_\theta(u)$ for the **uniform distribution**:

In [ ]:
# generator
def gen_unif(u):
  return u

Generator $G_\theta(u)$ and generator gradient $\nabla_\theta G_\theta(u)$ for the **Gaussian distribution**:

In [ ]:
# generator using Box-Muller transform
def gen_gaussian(n, d, unif, theta, sigma):

  unif[unif==0] = np.nextafter(0, 1)

  # if d is odd, add one dimension
  if d % 2 != 0:
    dim = d + 1
  else:
    dim = d

  # create standard normal samples
  u = np.zeros((n,dim))
  for i in np.arange(0,dim,2):
    u[:,i:(i+2)] = boxmuller(unif[:,i],unif[:,(i+1)])

  # if d is odd, drop one dimension
  if d % 2 != 0:
    u = np.delete(u,-1,1)

  # generate samples
  x = theta + u*sigma

  return x

# gradient of the generator
def grad_gen_gaussian(n, theta):
    return np.broadcast_to(np.expand_dims(np.eye(theta.shape[0]),axis=2),(theta.shape[0],theta.shape[0],n))

In [ ]:
# generator using inverse transform
def gen_gaussian_inv(n, d, unif, theta, sigma):

  unif[unif==0] = np.nextafter(0, 1)

  u = stats.norm.ppf(unif, loc=0, scale=1)

  # generate samples
  x = theta + u*sigma

  return x

# gradient of the generator
def grad_gen_gaussian(n, theta):
    return np.broadcast_to(np.expand_dims(np.eye(theta.shape[0]),axis=2),(theta.shape[0],theta.shape[0],n))

Generator $G_\theta(u)$ and generator gradient $\nabla_\theta G_\theta(u)$ for the **g-and-k distribution**:

In [ ]:
# generator
def gen_gandk(z, theta):
  a = theta[0]
  b = theta[1]
  g = theta[2]
  k = np.exp(theta[3])
  x = a+b*(1+0.8*((1-np.exp(-g*z))/(1+np.exp(-g*z))))*((1+z**2)**(k))*z
  return x

# gradient of the generator
def grad_gen_gandk(z,theta):
    a = theta[0]
    b = theta[1]
    g = theta[2]
    k = np.exp(theta[3])
    grad1 = np.ones(z.shape[0])
    grad2 = (1+(4/5)*((1-np.exp(-g*z))/(1+np.exp(-g*z))))*(np.exp(k*np.log(1+z**2)))*z
    grad3 = (8/5)*theta[1]*((np.exp(g*z))/(1+np.exp(g*z))**2)*(np.exp(k*np.log(1+z**2)))*z**2
    grad4 = b*(1+0.8*((1-np.exp(-g*z))/(1+np.exp(-g*z))))*(np.exp(k*np.log(1+z**2)))*np.log(1+z**2)*z
    return np.expand_dims(np.einsum('ij->ji',np.c_[grad1,grad2,grad3,grad4]), axis=0)

Generator $G_\theta(u)$ and its gradient $\nabla_\theta G_\theta(u)$ for **multivariate g-and-k distribution**:

In [ ]:
# generator that can be differentiated using jax
def gen_mvgandk_fct(z, theta):
  a = theta[0]
  b = theta[1]
  g = theta[2]
  k = theta[3]
  t = theta[4]

  d = z.shape[1]
  if d>1:
    # loop over i
    def body_fun2(i,sqrtsigma):
      # loop over j
      def body_fun1(j,sqrtsigma):
        # inner loop
        def body_fun(l,curr):
          curr = curr + jnp.sqrt(1+2*t*jnp.cos((l*jnp.pi)/(d+1)))*jnp.sin((i*l*jnp.pi)/(d+1))*jnp.sin((j*l*jnp.pi)/(d+1))
          return curr
        sqrtsigma = ops.index_update(sqrtsigma,ops.index[i-1,j-1],jnp.squeeze(lax.fori_loop(1, d+1, body_fun,jnp.array([0.]))))
        return sqrtsigma
      sqrtsigma = ops.index_update(sqrtsigma,ops.index[i-1,:],lax.fori_loop(1, d+1, body_fun1,sqrtsigma)[i-1,:])
      return sqrtsigma
    sqrtsigma = lax.fori_loop(1, d+1, body_fun2, jnp.zeros([d,d]))
    sqrtsigma = sqrtsigma*(2/(d+1))
    z = jnp.einsum('ij,lj->li',sqrtsigma,z)

  x = a+b*(1+0.8*((1-jnp.exp(-g*z))/(1+jnp.exp(-g*z))))*((1+z**2)**(k))*z
  return x

# generator to avoid numerical instabilities
def gen_mvgandk(z, theta):
  theta_gen = deepcopy(theta)
  theta_gen[3] = np.exp(theta[3])
  x = gen_mvgandk_fct(z, theta_gen)
  return x

# gradient of generator avoiding numerical instabilities
def grad_gen_mvgandk(z,theta):
  grad_fct = jacfwd(gen_mvgandk_fct,1)
  theta_gen = deepcopy(theta)
  theta_gen[3] = np.exp(theta[3])
  grad = grad_fct(z,theta_gen)
  return np.einsum('ijl->jli',grad) # dimensions: [d,p,n]

Generator $G_\theta(u)$ for the **bivariate beta distribution**:

In [ ]:
def sample_qmc_gamma(alpha,n):

  'Function to simulate from a gamma distribution with \alpha<1 and \beta=1 using QMC points'

  # get b
  b = (alpha+np.e)/np.e

  # fix QMC sequence
  qmc = qmcpy.Halton(3)

  # generate samples
  i = 0
  num = 0
  gamma = np.array([])
  while num < n:
    # get qmc samples
    omega = np.squeeze(qmc.gen_samples(n_min=i,n_max=i+1))
    #omega = np.squeeze(qmcpy.Halton(3).gen_samples(1))
    #omega = np.random.rand(3)
    # accept-reject algorihtm
    y = b*omega[0]
    if y<=1:
      x = y**(1/alpha)
      #print(x)
      w = -np.log(omega[1])
      #print(w)
      if w>=x:
        gamma = np.append(gamma,x)
        num += 1 # counts accepted points
    else:
      x = -np.log((b-y)/alpha)
      w = omega[2]**(1/(alpha-1))
      if w>=x:
        gamma = np.append(gamma,x)
        num += 1 # counts accepted points
    i += 1 # counts number of simulated points

  return gamma

In [ ]:
def sample_qmc_gamma(alpha,n):

  'Function to simulate from a gamma distribution with \alpha<1 and \beta=1 using QMC points'

  # get b
  b = (alpha+np.e)/np.e

  # fix QMC sequence
  qmc = qmcpy.Halton(3)

  # generate samples
  i = 0
  num = 0
  gamma = np.array([])
  while num < n:
    # get qmc samples
    omega = np.squeeze(qmc.gen_samples(n_min=i,n_max=i+1))
    # accept-reject algorihtm
    y = b*omega[0]
    if y<=1:
      x = y**(1/alpha)
      if omega[1]<=np.e**(-x):
        gamma = np.append(gamma,x)
        num += 1 # counts accepted points
    else:
      x = -np.log((b-y)/alpha)
      if omega[2]<=x**(alpha-1):
        gamma = np.append(gamma,x)
        num += 1 # counts accepted points
    i += 1 # counts number of simulated points

  return gamma

In [ ]:
def gen_bibeta(theta, unif, qmc_gamma=False):
  # number of samples
  n = unif.shape[0]

  # split theta into integer and decimal parts
  i_theta,d_theta = divmod(theta,np.ones(5)) 
  i_theta = i_theta.astype(int)

  # initialise
  utild = np.zeros([n,5])
  x = np.zeros((n,2))

  # logs on uniforms
  logunif = np.log(unif)

  # get \tilde{u}
  j=0
  for i in range(5):
    sum = np.zeros(n)
    if i_theta[i]!=0:
      for k in range(i_theta[i]):
        sum[:] += logunif[:,k+j]
      utild[:,i] = -sum
    if d_theta[i]!=0:
      if qmc_gamma:
        utild[:,i] += sample_qmc_gamma(d_theta[i],n)
      else:
        utild[:,i] += np.random.gamma(d_theta[i],1,n)
    j += i_theta[i]

  # generator
  x1 = np.sum(np.vstack([utild[:,0],utild[:,2]]),axis=0)/np.sum(np.vstack([utild[:,0],utild[:,2],utild[:,3],utild[:,4]]),axis=0)
  x2 = np.sum(np.vstack([utild[:,1],utild[:,3]]),axis=0)/np.sum(np.vstack([utild[:,1],utild[:,2],utild[:,3],utild[:,4]]),axis=0)

  return np.vstack([x1,x2]).T

## Sampling

Function to sample from **uniform distribution**:

In [ ]:
def sample_unif(method_sampling,n,d,sobol=False):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '
  ' as either Halton, Sobol or Lattice points                               '

  if method_sampling == 'MC':
    unif = np.random.rand(n,d)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    if sobol:
      unif = qmcpy.Sobol(d).gen_samples(n)
    else:
      unif = qmcpy.Halton(d).gen_samples(n)

  # generate samples
  x = gen_unif(unif)  

  return x

Function to sample $R$-times for MC and RQMC from **uniform distribution**:

In [ ]:
def sample_unif_r(n,num,d,lattice=False,order=1,sobol=False,z_path=None):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '
  ' as either Halton, Sobol or Lattice points                               '

  # generate n data points using MC and RQMC
  x_mc = np.zeros((num,np.max(n),d))
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d)
    if lattice:
      if order==1:
        unif_rqmc = qmcpy.Lattice(d).gen_samples(np.max(n))
      if order==2:
        unif_rqmc = qmcpy.Lattice(d,z_path=z_path+'lattice_vec.600.20.npy').gen_samples(np.max(n))
      if order==8:
        unif_rqmc = qmcpy.Lattice(d,z_path=z_path+'lattice_vec.600.13.npy').gen_samples(np.max(n))
    else:
      if sobol:
        unif_rqmc = qmcpy.Sobol(d).gen_samples(np.max(n))
      else:
        unif_rqmc = qmcpy.Halton(d).gen_samples(np.max(n))
    x_mc[r,:,:] = gen_unif(unif_mc)
    x_rqmc[r,:,:] = gen_unif(unif_rqmc)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  x_qmc = gen_unif(unif_qmc)

  return list([x_mc,x_qmc,x_rqmc])

Function to sample $R$-times for MC and RQMC with varying $d$ from **uniform distribution**:

In [ ]:
def sample_unif_r_d(n,num,d,lattice=False,order=1,sobol=False,z_path=None):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '
  ' as either Halton, Sobol or Lattice points                               '

  # generate n data points using MC and RQMC
  x_mc = np.zeros((num,n,np.max(d)))
  x_rqmc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    unif_mc = np.random.rand(n,np.max(d))
    if lattice:
      if order==1:
        unif_rqmc = qmcpy.Lattice(np.max(d)).gen_samples(n)
      if order==2:
        unif_rqmc = qmcpy.Lattice(np.max(d),z_path=z_path+'lattice_vec.600.20.npy').gen_samples(n)
      if order==8:
        unif_rqmc = qmcpy.Lattice(np.max(d),z_path=z_path+'lattice_vec.600.13.npy').gen_samples(n)
    else:
      if sobol:
        unif_rqmc = qmcpy.Sobol(np.max(d)).gen_samples(n)
      else:
        unif_rqmc = qmcpy.Halton(np.max(d)).gen_samples(n)
    x_mc[r,:,:] = gen_unif(unif_mc)
    x_rqmc[r,:,:] = gen_unif(unif_rqmc)

  # QMC
  unif_qmc = qmc.gen_samples(n)
  x_qmc = gen_unif(unif_qmc)

  return list([x_mc,x_qmc,x_rqmc])

Function to sample from **Gaussian distribution**:



In [ ]:
def sample_gaussian(method_sampling,n,d,s,theta,sobol=False):

  ' caveat:                                                                 '
  ' the qmc or qmc_1 sequence have to be fixed before using this function   '

  # odd number of parameters
  if d % 2 != 0: 
    if method_sampling == 'MC':
      unif = np.random.rand(n,d+1)
    if method_sampling == 'QMC':
      unif = qmc_1.gen_samples(n)
    if method_sampling == 'RQMC':
      if sobol:
        unif = qmcpy.Sobol(d+1).gen_samples(n)
      else:
        unif = qmcpy.Halton(d+1).gen_samples(n)

  # even number of parameters
  else: 
    if method_sampling == 'MC':
      unif = np.random.rand(n,d)
    if method_sampling == 'QMC':
      unif = qmc.gen_samples(n)
    if method_sampling == 'RQMC':
      if sobol:
        unif = qmcpy.Sobol(d).gen_samples(n)
      else:
        unif = qmcpy.Halton(d).gen_samples(n)

  # use generator  
  x = gen_gaussian(n,d,unif,theta,s)

  return x

In [ ]:
def sample_gaussian_inv(method_sampling,n,d,s,theta,sobol=False):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function   '

  if method_sampling == 'MC':
    unif = np.random.rand(n,d)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    if sobol:
      unif = qmcpy.Sobol(d).gen_samples(n)
    else:
      unif = qmcpy.Halton(d).gen_samples(n)

  # use generator  
  x = gen_gaussian_inv(n,d,unif,theta,s)

  return x

Function to sample $R$-times for MC and RQMC from **Gaussian distribution**:

In [ ]:
def sample_gaussian_r(n,num,d,s,theta,lattice=False,order=1,sobol=False,z_path=None):

  ' caveat:                                                                 '
  ' the qmc or qmc_1 sequence have to be fixed before using this function   '

  # MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    # odd number of parameters
    if d % 2 != 0: 
      unif_mc = np.random.rand(np.max(n),d+1)
    # even number of parameters
    else: 
      unif_mc = np.random.rand(np.max(n),d)
    x = gen_gaussian(np.max(n),d,unif_mc,theta,s)
    x_mc[r,:,:] = x

  # QMC
  # odd number of parameters
  if d % 2 != 0: 
    unif_qmc = qmc_1.gen_samples(np.max(n))
  # even number of parameters
  else: 
    unif_qmc = qmc.gen_samples(np.max(n))
  x_qmc = gen_gaussian(np.max(n),d,unif_qmc,theta,s)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    # odd number of parameters
    if d % 2 != 0: 
      if lattice:
        if order==1:
          unif_rqmc = qmcpy.Lattice(d+1).gen_samples(np.max(n))
        if order==2:
          unif_rqmc = qmcpy.Lattice(d+1,z_path=z_path+'lattice_vec.600.20.npy').gen_samples(np.max(n))
        if order==8:
          unif_rqmc = qmcpy.Lattice(d+1,z_path=z_path+'lattice_vec.600.13.npy').gen_samples(np.max(n))
      else:
        if sobol:
          unif_rqmc = qmcpy.Sobol(d+1).gen_samples(np.max(n))
        else:
          unif_rqmc = qmcpy.Halton(d+1).gen_samples(np.max(n))
    # even number of parameters
    else: 
      if lattice:
        if order==1:
          unif_rqmc = qmcpy.Lattice(d).gen_samples(np.max(n))
        if order==2:
          unif_rqmc = qmcpy.Lattice(d,z_path=z_path+'lattice_vec.600.20.npy').gen_samples(np.max(n))
        if order==8:
          unif_rqmc = qmcpy.Lattice(d,z_path=z_path+'lattice_vec.600.13.npy').gen_samples(np.max(n))
      else:
        if sobol:
          unif_rqmc = qmcpy.Sobol(d).gen_samples(np.max(n))
        else:
          unif_rqmc = qmcpy.Halton(d).gen_samples(np.max(n))
        
    x = gen_gaussian(np.max(n),d,unif_rqmc,theta,s)
    x_rqmc[r,:,:] = x

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_gaussian_r_d(n,num,d,s,theta,lattice=False,order=1,sobol=False, z_path=None):

  ' caveat:                                                                 '
  ' the qmc or qmc_1 sequence have to be fixed before using this function   '

  # MC
  x_mc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    # odd number of parameters
    if np.max(d) % 2 != 0: 
      unif_mc = np.random.rand(n,np.max(d)+1)
    # even number of parameters
    else: 
      unif_mc = np.random.rand(n,np.max(d))
    x = gen_gaussian(n,np.max(d),unif_mc,theta,s)
    x_mc[r,:,:] = x

  # QMC
  # odd number of parameters
  if np.max(d) % 2 != 0: 
    unif_qmc = qmc_1.gen_samples(n)
  # even number of parameters
  else: 
    unif_qmc = qmc.gen_samples(n)
  x_qmc = gen_gaussian(n,np.max(d),unif_qmc,theta,s)

  # RQMC
  x_rqmc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    # odd number of parameters
    if np.max(d) % 2 != 0: 
      if lattice:
        if order==1:
          unif_rqmc = qmcpy.Lattice(np.max(d)+1).gen_samples(n)
        if order==2:
          unif_rqmc = qmcpy.Lattice(np.max(d)+1,z_path=z_path+'lattice_vec.600.20.npy').gen_samples(n)
        if order==8:
          unif_rqmc = qmcpy.Lattice(np.max(d)+1,z_path=z_path+'lattice_vec.600.13.npy').gen_samples(n)
      else:
        if sobol:
          unif_rqmc = qmcpy.Sobol(np.max(d)+1).gen_samples(n)
        else:
          unif_rqmc = qmcpy.Halton(np.max(d)+1).gen_samples(n)
    # even number of parameters
    else: 
      if lattice:
        if order==1:
          unif_rqmc = qmcpy.Lattice(np.max(d)).gen_samples(n)
        if order==2:
          unif_rqmc = qmcpy.Lattice(np.max(d),z_path=z_path+'lattice_vec.600.20.npy').gen_samples(n)
        if order==8:
          unif_rqmc = qmcpy.Lattice(np.max(d),z_path=z_path+'lattice_vec.600.13.npy').gen_samples(n)
      else:
        if sobol:
          unif_rqmc = qmcpy.Sobol(np.max(d)).gen_samples(n)
        else:
          unif_rqmc = qmcpy.Halton(np.max(d)).gen_samples(n)
    x = gen_gaussian(n,np.max(d),unif_rqmc,theta,s)
    x_rqmc[r,:,:] = x

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_gaussian_r_inv(n,num,d,s,theta,lattice=False,order=1,sobol=False,z_path=None):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function   '

  # MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d)
    x = gen_gaussian_inv(np.max(n),d,unif_mc,theta,s)
    x_mc[r,:,:] = x

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  x_qmc = gen_gaussian_inv(np.max(n),d,unif_qmc,theta,s)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    if lattice:
      if order==1:
        unif_rqmc = qmcpy.Lattice(d).gen_samples(np.max(n))
      if order==2:
        unif_rqmc = qmcpy.Lattice(d,z_path=z_path+'lattice_vec.600.20.npy').gen_samples(np.max(n))
      if order==8:
        unif_rqmc = qmcpy.Lattice(d,z_path=z_path+'lattice_vec.600.13.npy').gen_samples(np.max(n))
    else:
      if sobol:
        unif_rqmc = qmcpy.Sobol(d).gen_samples(np.max(n))
      else:
        unif_rqmc = qmcpy.Halton(d).gen_samples(np.max(n))
    x = gen_gaussian_inv(np.max(n),d,unif_rqmc,theta,s)
    x_rqmc[r,:,:] = x

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_gaussian_r_inv_d(n,num,d,s,theta,lattice=False,order=1,sobol=False,z_path=None):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function   '

  # MC
  x_mc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    unif_mc = np.random.rand(n,np.max(d))
    x = gen_gaussian_inv(n,np.max(d),unif_mc,theta,s)
    x_mc[r,:,:] = x

  # QMC
  unif_qmc = qmc.gen_samples(n)
  x_qmc = gen_gaussian_inv(n,np.max(d),unif_qmc,theta,s)

  # RQMC
  x_rqmc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    if lattice:
      if order==1:
        unif_rqmc = qmcpy.Lattice(np.max(d)).gen_samples(n)
      if order==2:
        unif_rqmc = qmcpy.Lattice(np.max(d),z_path=z_path+'lattice_vec.600.20.npy').gen_samples(n)
      if order==8:
        unif_rqmc = qmcpy.Lattice(np.max(d),z_path=z_path+'lattice_vec.600.13.npy').gen_samples(n)
    else:
      if sobol:
        unif_rqmc = qmcpy.Sobol(np.max(d)).gen_samples(n)
      else:
        unif_rqmc = qmcpy.Halton(np.max(d)).gen_samples(n)
    x = gen_gaussian_inv(n,np.max(d),unif_rqmc,theta,s)
    x_rqmc[r,:,:] = x

  return list([x_mc,x_qmc,x_rqmc])

Function to sample from **g-and-k distribution**:

In [ ]:
def sample_gandk(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # generate uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,d+1)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(d+1).gen_samples(n)

  # generate standard normals  
  z = normals(n,d,unif)

  # generate samples from g-and-k distribution
  x = gen_gandk(z,theta)

  return list([x,z])

In [ ]:
def sample_gandk_inv(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # generate uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,d)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(d).gen_samples(n)

  # generate standard normals  
  z = normals_inv(unif)

  # generate samples from g-and-k distribution
  x = gen_gandk(z,theta)

  return list([x,z])

Function to sample from **multivariate g-and-k distribution**:

In [ ]:
def sample_mvgandk(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # generate uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,d+1)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(d+1).gen_samples(n)

  # generate standard normals  
  z = normals(n,d,unif)

  # generate samples from g-and-k distribution
  x = gen_mvgandk(z,theta)

  return list([x,z])

In [ ]:
def sample_mvgandk_inv(method_sampling,n,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # generate uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,d)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(d).gen_samples(n)

  # generate standard normals  
  z = normals_inv(unif)

  # generate samples from g-and-k distribution
  x = gen_mvgandk(z,theta)

  return list([x,z])

Function to sample $R$-times for MC and RQMC from **g-and-k distribution**:

In [ ]:
def sample_gandk_r(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d+1)
    z = normals(np.max(n), d, unif_mc)
    x_mc[r,:,:] = gen_gandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  z = normals(np.max(n), d, unif_qmc)
  x_qmc = gen_gandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(d+1).gen_samples(np.max(n))
    z = normals(np.max(n), d, unif_rqmc)
    x_rqmc[r,:,:] = gen_gandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_gandk_r_inv(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d)
    z = normals_inv(unif_mc)
    x_mc[r,:,:] = gen_gandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  z = normals_inv(unif_qmc)
  x_qmc = gen_gandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(d).gen_samples(np.max(n))
    z = normals_inv(unif_rqmc)
    x_rqmc[r,:,:] = gen_gandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

Function to sample $R$-times for MC and RQMC from **multivariate g-and-k distribution**:

In [ ]:
def sample_mvgandk_r(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d+1)
    z = normals(np.max(n), d, unif_mc)
    x_mc[r,:,:] = gen_mvgandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  z = normals(np.max(n), d, unif_qmc)
  x_qmc = gen_mvgandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(d+1).gen_samples(np.max(n))
    z = normals(np.max(n), d, unif_rqmc)
    x_rqmc[r,:,:] = gen_mvgandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_mvgandk_r_inv(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_mc = np.random.rand(np.max(n),d)
    z = normals_inv(unif_mc)
    x_mc[r,:,:] = gen_mvgandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(n))
  z = normals_inv(unif_qmc)
  x_qmc = gen_mvgandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),d))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(d).gen_samples(np.max(n))
    z = normals_inv(unif_rqmc)
    x_rqmc[r,:,:] = gen_mvgandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_mvgandk_r_d(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    unif_mc = np.random.rand(n,np.max(d)+1)
    z = normals(n, np.max(d), unif_mc)
    x_mc[r,:,:] = gen_mvgandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(np.max(d)+1)
  z = normals(n, np.max(d), unif_qmc)
  x_qmc = gen_mvgandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(np.max(d)+1).gen_samples(n)
    z = normals(n, np.max(d), unif_rqmc)
    x_rqmc[r,:,:] = gen_mvgandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

In [ ]:
def sample_mvgandk_r_inv_d(n,num,d,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  #MC
  x_mc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    unif_mc = np.random.rand(n,np.max(d))
    z = normals_inv(unif_mc)
    x_mc[r,:,:] = gen_mvgandk(z,theta)

  # QMC
  unif_qmc = qmc.gen_samples(n)
  z = normals_inv(unif_qmc)
  x_qmc = gen_mvgandk(z,theta)

  # RQMC
  x_rqmc = np.zeros((num,n,np.max(d)))
  for r in range(num):
    unif_rqmc = qmcpy.Halton(np.max(d)).gen_samples(n)
    z = normals_inv(unif_rqmc)
    x_rqmc[r,:,:] = gen_mvgandk(z,theta)

  return list([x_mc,x_qmc,x_rqmc])

Function to sample from **bivariate beta distribution**:

In [ ]:
def sample_bibeta(method_sampling,n,theta):

  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # split theta into integer and decimal parts
  i_theta,_ = divmod(theta,np.ones(5)) 
  p = np.sum(i_theta.astype(int))

  # sample uniforms
  if method_sampling == 'MC':
    unif = np.random.rand(n,p)
  if method_sampling == 'QMC':
    unif = qmc.gen_samples(n)
  if method_sampling == 'RQMC':
    unif = qmcpy.Halton(p).gen_samples(n)

  # generate samples
  if method_sampling == 'QMC' or method_sampling == 'RQMC':
    x = gen_bibeta(theta,unif,qmc_gamma=True)
  else: 
    x = gen_bibeta(theta,unif)

  return x

Funktion to sample $R$-times for MC and RQMC from **bivariate beta distribution**:

In [ ]:
 def sample_bibeta_r(n,num,theta):
  ' caveat:                                                                 '
  ' the qmc sequence has to be fixed before using this function             '

  # split theta into integer and decimal parts
  i_theta,_ = divmod(theta,np.ones(5)) 
  p = np.sum(i_theta.astype(int))

  #MC
  x_mc = np.zeros((num,np.max(n),2))
  for r in range(num):
    unif = np.random.rand(np.max(n),p)
    x_mc[r,:,:] = gen_bibeta(theta,unif)

  # QMC
  unif = qmc.gen_samples(np.max(n))
  x_qmc = gen_bibeta(theta,unif,qmc_gamma=True)

  # RQMC
  x_rqmc = np.zeros((num,np.max(n),2))
  for r in range(num):
    unif = qmcpy.Halton(p).gen_samples(np.max(n))
    x_rqmc[r,:,:] = gen_bibeta(theta,unif,qmc_gamma=True)

  return list([x_mc,x_qmc,x_rqmc])

## Optimisation loop

In [ ]:
def optim(model,method_sampling,method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel='gaussian',s=2,sparse=False):

  ' caveat:                                                                '
  ' the qmc sequence has to be fixed before using this function            '
  
  ' function arguments:                                                    '   
  ' model:             "gaussian" or "gandk" or "sv"                       '
  ' method_sampling:   "MC" or "QMC" or "RQMC"                             '
  ' method_sg:         "SGD" or "NSGD"                                     '
  ' eta:               step size                                           '
  ' max_it:            maximum number of iterations                        '
  ' l:                 lengthscale of the Gaussian kernel                  '
  ' c:                 parameter c of the IMQ kernel                       '
  ' b:                 parameter beta of the IMQ kernel                    '
  ' n:                 number of samples simulated per iteration           '
  ' m:                 number of true samples                              '
  ' d:                 number of dimensions                                '
  ' y:                 true data set                                       '
  ' start:             start values                                        '
  ' kernel (optional): "gaussian" or "imq"                                 '
  ' s (optional):      standard deviation in Gaussian location model       '
  ' sparse (optional): True or False                                       '

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(y,y,'sqeuclidean')))

  # pre-define noise for information metric
  noise = [0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

  # pre-compute the kernel and its derivatives for the true data points
  kyy = k(y,y,kernel,l,c,b,nu,grad=False,sparse=True)
  
  # list for squared MMD
  loss = []  

  # start values
  theta = np.expand_dims(start,axis=0) 

  # create timer instance
  t = TicToc()
  t.tic()

  for i in range(max_it):
    
    # simulate data using the current estimate for theta
    if model == 'gaussian':
      x = sample_gaussian(method_sampling,n,d,s,theta[i,:])
    if model == 'gandk':
      x,z = sample_gandk(method_sampling,n,d,theta[i,:])
    
    # calculate kernel and the derivatives
    kxx = k(x,x,kernel,l,c,b,nu,sparse=False)
    kxy = k(x,y,kernel,l,c,b,nu,sparse=False)
    
    # calculate the gradient of the generator
    if model == 'gaussian':
      grad_g = grad_gen_gaussian(n,theta[i,:])
    if model == 'gandk':
      grad_g = grad_gen_gandk(z, theta[i,:])
    
    # approximate squared MMD gradient
    if p==1:
      J = np.asmatrix(grad_MMD(p,n,m,grad_g,kxx[1],kxy[1],stat_type))
    else:
      J = grad_MMD(p,n,m,grad_g,kxx[1],kxy[1],stat_type)
    
    # approximate information metric
    if method_gd == 'NSGD':
      g = g_approx(p,n,grad_g,kxx[2])
      # add noise if g can't be inverted
      for j in range(9):
        check = True
        try:
          np.linalg.inv(g + np.eye(p)*noise[j])
        except np.linalg.LinAlgError:
          check = False
        if check:
          break
      g = g + np.eye(p)*noise[j]
    
    # update estimate for theta using NSGD or SGD
    if method_gd == 'NSGD':
        theta = np.vstack([theta,theta[i,:]-eta*np.linalg.inv(g)@J]) # NSGD
    else:
        theta = np.vstack([theta,theta[i,:]-eta*J]) # SGD
    
    # calculate current squared MMD approximation
    loss.append(MMD_approx(n,m,kxx[0],kxy[0],kyy[0],stat_type))
    
    # print outputs
    if (i+1)%1000 == 0:
        print('iteration:',i+1,'\nloss:     ', round(loss[i],7),'\nestimate: ',theta[i+1,:],'\ngradient: ', J)

    # stop if nan occurs
    if np.isnan(loss[i]):
      break

  print('-------------------------------------------\nfinal loss:       ', round(loss[i],7), '\nfinal estimate:   ', theta[i+1,:],'\ntotal iterations: ',i+1)
  t.toc() 

  np.savetxt(fname=method_sampling+'_theta.csv', delimiter=",", X=theta)
  np.savetxt(fname=method_sampling+'_loss.csv', delimiter=",", X=loss)

  return list([theta, loss])


In [ ]:
def optim_inv_nosub(model,method_sampling,method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel='gaussian',s=2):

  ' function uses inverse transform instead of Box-Muller transform        '

  ' caveat:                                                                '
  ' the qmc sequence has to be fixed before using this function            '
  
  ' function arguments:                                                    '   
  ' model:             "gaussian" or "gandk" or "sv"                       '
  ' method_sampling:   "MC" or "QMC" or "RQMC"                             '
  ' method_sg:         "SGD" or "NSGD"                                     '
  ' eta:               step size                                           '
  ' max_it:            maximum number of iterations                        '
  ' l:                 lengthscale of the Gaussian kernel                  '
  ' c:                 parameter c of the IMQ kernel                       '
  ' b:                 parameter beta of the IMQ kernel                    '
  ' n:                 number of samples simulated per iteration           '
  ' m:                 number of true samples                              '
  ' d:                 number of dimensions                                '
  ' y:                 true data set                                       '
  ' start:             start values                                        '
  ' kernel (optional): "gaussian" or "imq"                                 '
  ' s (optional):      standard deviation in Gaussian location model       '
  ' sparse (optional): True or False                                       '

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(y,y,'sqeuclidean')))

  # pre-define noise for information metric
  noise = [0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

  # pre-compute the kernel and its derivatives for the true data points
  kyy = k(y,y,kernel,l,c,b,nu,grad=False,sparse=True)
  
  # list for squared MMD
  loss = []  

  # start values
  theta = np.expand_dims(start,axis=0) 

  # create timer instance
  t = TicToc()
  t.tic()

  for i in range(max_it):
    
    # simulate data using the current estimate for theta
    if model == 'gaussian':
      x = sample_gaussian_inv(method_sampling,n,d,s,theta[i,:])
    if model == 'gandk':
      x,z = sample_gandk_inv(method_sampling,n,d,theta[i,:])
    if model == 'mvgandk':
      x,z = sample_mvgandk_inv(method_sampling,n,d,theta[i,:])
    
    # calculate kernel and the derivatives
    kxx = k(x,x,kernel,l,c,b,nu,sparse=False)
    kxy = k(x,y,kernel,l,c,b,nu,sparse=False)
    
    # calculate the gradient of the generator
    if model == 'gaussian':
      grad_g = grad_gen_gaussian(n,theta[i,:])
    if model == 'gandk':
      grad_g = grad_gen_gandk(z, theta[i,:])
    if model == 'mvgandk':
      grad_g = grad_gen_mvgandk(z, theta[i,:])
    
    # approximate squared MMD gradient
    if p==1:
      J = np.asmatrix(grad_MMD(p,n,m,grad_g,kxx[1],kxy[1],stat_type))
    else:
      J = grad_MMD(p,n,m,grad_g,kxx[1],kxy[1],stat_type)
    
    # approximate information metric
    if method_gd == 'NSGD':
      g = g_approx(p,n,grad_g,kxx[2])
      # add noise if g can't be inverted
      for j in range(9):
        check = True
        try:
          np.linalg.inv(g + np.eye(p)*noise[j])
        except np.linalg.LinAlgError:
          check = False
        if check:
          break
      g = g + np.eye(p)*noise[j]
    
    # update estimate for theta using NSGD or SGD
    if method_gd == 'NSGD':
        theta = np.vstack([theta,theta[i,:]-eta*np.linalg.inv(g)@J]) # NSGD
    else:
        theta = np.vstack([theta,theta[i,:]-eta*J]) # SGD
    
    # calculate current squared MMD approximation
    loss.append(MMD_approx(n,m,kxx[0],kxy[0],kyy[0],stat_type))
    
    # print outputs
    if (i+1)%1000 == 0:
        print('iteration:',i+1,'\nloss:     ', round(loss[i],7),'\nestimate: ',theta[i+1,:],'\ngradient: ', J)

    # stop if nan occurs
    if np.isnan(loss[i]):
      break

  print('-------------------------------------------\nfinal loss:       ', round(loss[i],7), '\nfinal estimate:   ', theta[i+1,:],'\ntotal iterations: ',i+1)
  t.toc() 

  np.savetxt(fname=method_sampling+'_theta.csv', delimiter=",", X=theta)
  np.savetxt(fname=method_sampling+'_loss.csv', delimiter=",", X=loss)

  return list([theta, loss])

In [ ]:
def optim_inv(model,method_sampling,method_gd,eta,max_it,l,c,b,nu,n,m,d,p,y,start,stat_type,kernel='gaussian',s=2):

  ' function uses inverse transform instead of Box-Muller transform        '

  ' caveat:                                                                '
  ' the qmc sequence has to be fixed before using this function            '
  
  ' function arguments:                                                    '   
  ' model:             "gaussian" or "gandk" or "sv"                       '
  ' method_sampling:   "MC" or "QMC" or "RQMC"                             '
  ' method_sg:         "SGD" or "NSGD"                                     '
  ' eta:               step size                                           '
  ' max_it:            maximum number of iterations                        '
  ' l:                 lengthscale of the Gaussian kernel                  '
  ' c:                 parameter c of the IMQ kernel                       '
  ' b:                 parameter beta of the IMQ kernel                    '
  ' n:                 number of samples simulated per iteration           '
  ' m:                 number of true samples                              '
  ' d:                 number of dimensions                                '
  ' y:                 true data set                                       '
  ' start:             start values                                        '
  ' kernel (optional): "gaussian" or "imq"                                 '
  ' s (optional):      standard deviation in Gaussian location model       '
  ' sparse (optional): True or False                                       '

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(y,y,'sqeuclidean')))

  # pre-define noise for information metric
  noise = [0, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]
  
  # list for squared MMD
  loss = []  

  # start values
  theta = np.expand_dims(start,axis=0) 

  # create timer instance
  t = TicToc()
  t.tic()

  for i in range(max_it):

    # subsample from the true data
    sub = np.random.choice(np.arange(y.shape[0]),m)
    y_sub = y[sub,:]
    
    # simulate data using the current estimate for theta
    if model == 'gaussian':
      x = sample_gaussian_inv(method_sampling,n,d,s,theta[i,:])
    if model == 'gandk':
      x,z = sample_gandk_inv(method_sampling,n,d,theta[i,:])
    if model == 'mvgandk':
      x,z = sample_mvgandk_inv(method_sampling,n,d,theta[i,:])
    
    # calculate kernel and the derivatives
    kyy = k(y_sub,y_sub,kernel,l,c,b,nu,grad=False,sparse=False)
    kxx = k(x,x,kernel,l,c,b,nu,sparse=False)
    kxy = k(x,y_sub,kernel,l,c,b,nu,sparse=False)
    
    # calculate the gradient of the generator
    if model == 'gaussian':
      grad_g = grad_gen_gaussian(n,theta[i,:])
    if model == 'gandk':
      grad_g = grad_gen_gandk(z, theta[i,:])
    if model == 'mvgandk':
      grad_g = grad_gen_mvgandk(z, theta[i,:])
    
    # approximate squared MMD gradient
    if p==1:
      J = np.asmatrix(grad_MMD(p,n,m,grad_g,kxx[1],kxy[1],stat_type))
    else:
      J = grad_MMD(p,n,m,grad_g,kxx[1],kxy[1],stat_type)
    
    # approximate information metric
    if method_gd == 'NSGD':
      g = g_approx(p,n,grad_g,kxx[2])
      # add noise if g can't be inverted
      for j in range(9):
        check = True
        try:
          np.linalg.inv(g + np.eye(p)*noise[j])
        except np.linalg.LinAlgError:
          check = False
        if check:
          break
      g = g + np.eye(p)*noise[j]
    
    # update estimate for theta using NSGD or SGD
    if method_gd == 'NSGD':
        theta = np.vstack([theta,theta[i,:]-eta*np.linalg.inv(g)@J]) # NSGD
    else:
        theta = np.vstack([theta,theta[i,:]-eta*J]) # SGD
    
    # calculate current squared MMD approximation
    loss.append(MMD_approx(n,m,kxx[0],kxy[0],kyy[0],stat_type))
    
    # print outputs
    if (i+1)%1000 == 0:
        print('iteration:',i+1,'\nloss:     ', round(loss[i],7),'\nestimate: ',theta[i+1,:],'\ngradient: ', J)

    # stop if nan occurs
    if np.isnan(loss[i]):
      break

  print('-------------------------------------------\nfinal loss:       ', round(loss[i],7), '\nfinal estimate:   ', theta[i+1,:],'\ntotal iterations: ',i+1)
  t.toc() 

  np.savetxt(fname=method_sampling+'_theta.csv', delimiter=",", X=theta)
  np.savetxt(fname=method_sampling+'_loss.csv', delimiter=",", X=loss)

  return list([theta, loss])

### MSE

In [ ]:
def mse(max_it,p,theta1,theta3,theta_star):
  mse1 = np.zeros((max_it-1,p))
  mse3 = np.zeros((max_it-1,p))
  for l in range(p):
    for j in range(max_it-1):
      mse1[j,l] = np.mean(np.asarray((theta1[1:j+2,l]-theta_star[l]))**2)
      mse3[j,l] = np.mean(np.asarray((theta3[1:j+2,l]-theta_star[l]))**2)
  return list([mse1,mse3])

## Convergence of MMD$^2$

Function to calculate MMD$^2$ against $n$ for uniform distribution, Gaussian distribution, g-and-k distribution and bivariate beta distribution:

In [ ]:
def mmd_conv(model,n,num,d,l,c,b,nu,kernel='gaussian',stat_type='v',theta=None,s=2,mc_all=False,lattice=False,order=1,sobol=False,z_path=None):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,lattice,order,sobol,z_path)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r(n,num,d,s,theta,lattice,order,sobol,z_path)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r(n,num,d,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r(n,num,d,theta)
  
  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(x_qmc,x_qmc,'sqeuclidean')))

  # calculate squared MMD for a sequence of n
  MMD_mc = []
  MMD_qmc = []
  MMD_rqmc = []
  MMD_min_mc = []
  MMD_max_mc = []
  MMD_min_rqmc = []
  MMD_max_rqmc = []
  MMD_all_mc = []
  for j in n:

    # R repetitions for MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      mc_ave.append(MMD_approx(j,j,k(x_mc[r,:j,:],x_mc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_mc[r,:j,:],y_mc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_mc[r,:j,:],y_mc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
      rqmc_ave.append(MMD_approx(j,j,k(x_rqmc[r,:j,:],x_rqmc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_rqmc[r,:j,:],y_rqmc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_rqmc[r,:j,:],y_rqmc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))

    if mc_all:
      # append all values for MC
      MMD_all_mc.append(np.array(mc_ave))

    # append min and max values for MC and RQMC
    MMD_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    MMD_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    MMD_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    MMD_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave))))

    # append value for QMC and mean values for MC and RQMC
    MMD_mc.append(np.mean(np.abs(np.array(mc_ave))))
    MMD_qmc.append(MMD_approx(j,j,k(x_qmc[:j,:],x_qmc[:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_qmc[:j,:],y_qmc[:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_qmc[:j,:],y_qmc[:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
    MMD_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    print('sample size: ', j)

  if mc_all:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc,MMD_all_mc])
  else:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc])

In [ ]:
def mmd_conv_inv(model,n,num,d,l,c,b,nu,kernel,stat_type,theta=None,s=2,mc_all=False,lattice=False,order=1,sobol=False,z_path=None):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,lattice,order,sobol,z_path)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,lattice,order,sobol,z_path)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_inv(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_inv(n,num,d,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r_inv(n,num,d,theta)

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(x_qmc,x_qmc,'sqeuclidean')))

  # calculate squared MMD for a sequence of n
  MMD_mc = []
  MMD_qmc = []
  MMD_rqmc = []
  MMD_min_mc = []
  MMD_max_mc = []
  MMD_min_rqmc = []
  MMD_max_rqmc = []
  MMD_all_mc = []
  for j in n:

    # R repetitions for MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      mc_ave.append(MMD_approx(j,j,k(x_mc[r,:j,:],x_mc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_mc[r,:j,:],y_mc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_mc[r,:j,:],y_mc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
      rqmc_ave.append(MMD_approx(j,j,k(x_rqmc[r,:j,:],x_rqmc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_rqmc[r,:j,:],y_rqmc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_rqmc[r,:j,:],y_rqmc[r,:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type)) 
    
    if mc_all:
      # append all values for MC
      MMD_all_mc.append(np.array(mc_ave))

    # append min and max values for MC and RQMC
    MMD_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    MMD_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    MMD_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    MMD_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave))))

    # append value for QMC and mean values for MC and RQMC
    MMD_mc.append(np.mean(np.abs(np.array(mc_ave))))
    MMD_qmc.append(MMD_approx(j,j,k(x_qmc[:j,:],x_qmc[:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_qmc[:j,:],y_qmc[:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_qmc[:j,:],y_qmc[:j,:],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
    MMD_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    print('sample size: ', j)

  if mc_all:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc,MMD_all_mc])
  else:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc])

Function to calculate MMD$^2$ against $d$ with fixed $n$ for uniform distribution, Gaussian distribution and g-and-k distribution:

In [ ]:
def mmd_conv_d(model,n,num,d,l,c,b,nu,kernel,stat_type,theta=None,s=2,mc_all=False,ladapt=False,lattice=False,order=1,sobol=False,z_path=None):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,lattice,order,sobol,z_path)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_d(n,num,d,s,theta,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_d(n,num,d,s,theta,lattice,order,sobol,z_path)
  if model=='mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_d(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_d(n,num,d,theta)

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(x_qmc,x_qmc,'sqeuclidean')))

  # calculate squared MMD for a sequence of n
  MMD_mc = []
  MMD_qmc = []
  MMD_rqmc = []
  MMD_min_mc = []
  MMD_max_mc = []
  MMD_min_rqmc = []
  MMD_max_rqmc = []
  MMD_all_mc = []
  for j in d:

    # R repetitions for MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      if ladapt:
        mc_ave.append(MMD_approx(n,n,k(x_mc[r,:,:j],x_mc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(x_mc[r,:,:j],y_mc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(y_mc[r,:,:j],y_mc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],stat_type))
        rqmc_ave.append(MMD_approx(n,n,k(x_rqmc[r,:,:j],x_rqmc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(x_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(y_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],stat_type)) 
      else:
        mc_ave.append(MMD_approx(n,n,k(x_mc[r,:,:j],x_mc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_mc[r,:,:j],y_mc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_mc[r,:,:j],y_mc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
        rqmc_ave.append(MMD_approx(n,n,k(x_rqmc[r,:,:j],x_rqmc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type)) 

    if mc_all:
      # append all values for MC
      MMD_all_mc.append(np.array(mc_ave))

    # append min and max values for MC and RQMC
    MMD_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    MMD_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    MMD_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    MMD_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave))))

    # append value for QMC and mean values for MC and RQMC
    MMD_mc.append(np.mean(np.abs(np.array(mc_ave))))
    if ladapt:
      MMD_qmc.append(MMD_approx(n,n,k(x_qmc[:,:j],x_qmc[:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(x_qmc[:,:j],y_qmc[:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(y_qmc[:,:j],y_qmc[:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],stat_type))
    else:
      MMD_qmc.append(MMD_approx(n,n,k(x_qmc[:,:j],x_qmc[:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_qmc[:,:j],y_qmc[:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_qmc[:,:j],y_qmc[:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
    MMD_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    print('number of dimensions: ', j)

  if mc_all:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc,MMD_all_mc])
  else:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc])

In [ ]:
def mmd_conv_inv_d(model,n,num,d,l,c,b,nu,kernel,stat_type,theta=None,s=2,mc_all=False,ladapt=False,lattice=False,order=1,sobol=False,z_path=None):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,lattice,order,sobol,z_path)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,lattice,order,sobol,z_path)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,lattice,order,sobol,z_path)
  if model=='mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_inv_d(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_inv_d(n,num,d,theta)

  # median heuristic if l=-1
  if l == -1:
    l = np.sqrt((1/2)*np.median(distance.cdist(x_qmc,x_qmc,'sqeuclidean')))

  # calculate squared MMD for a sequence of n
  MMD_mc = []
  MMD_qmc = []
  MMD_rqmc = []
  MMD_min_mc = []
  MMD_max_mc = []
  MMD_min_rqmc = []
  MMD_max_rqmc = []
  MMD_all_mc = []
  for j in d:

    # R repetitions for MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      if ladapt:
        mc_ave.append(MMD_approx(n,n,k(x_mc[r,:,:j],x_mc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(x_mc[r,:,:j],y_mc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(y_mc[r,:,:j],y_mc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],stat_type))
        rqmc_ave.append(MMD_approx(n,n,k(x_rqmc[r,:,:j],x_rqmc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(x_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(y_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],stat_type)) 
      else:
        mc_ave.append(MMD_approx(n,n,k(x_mc[r,:,:j],x_mc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_mc[r,:,:j],y_mc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_mc[r,:,:j],y_mc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
        rqmc_ave.append(MMD_approx(n,n,k(x_rqmc[r,:,:j],x_rqmc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_rqmc[r,:,:j],y_rqmc[r,:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type)) 
    
    if mc_all:
      # append all values for MC
      MMD_all_mc.append(np.array(mc_ave))

    # append min and max values for MC and RQMC
    MMD_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    MMD_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    MMD_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    MMD_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave))))

    # append value for QMC and mean values for MC and RQMC
    MMD_mc.append(np.mean(np.abs(np.array(mc_ave))))
    if ladapt:
      MMD_qmc.append(MMD_approx(n,n,k(x_qmc[:,:j],x_qmc[:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(x_qmc[:,:j],y_qmc[:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],k(y_qmc[:,:j],y_qmc[:,:j],kernel,l*np.sqrt(j),c,b,nu,grad=False,sparse=True)[0],stat_type))
    else:
      MMD_qmc.append(MMD_approx(n,n,k(x_qmc[:,:j],x_qmc[:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(x_qmc[:,:j],y_qmc[:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],k(y_qmc[:,:j],y_qmc[:,:j],kernel,l,c,b,nu,grad=False,sparse=True)[0],stat_type))
    MMD_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    print('number of dimensions: ', j)

  if mc_all:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc,MMD_all_mc])
  else:
    return list([MMD_mc,MMD_qmc,MMD_rqmc,MMD_min_mc,MMD_max_mc,MMD_min_rqmc,MMD_max_rqmc])

### Convergence of Wasserstein distance

Function to calculate Wasserstein distance against $n$ for uniform distribution, Gaussian distribution, g-and-k distribution and bivariate beta distribution:

In [ ]:
 def W_conv(model,n,num,d,theta=None,s=2,cost='euclidean',sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r(n,num,d,s,theta,sobol=sobol)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_gandk_r(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in n:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((j,)) / j 
    b = np.ones((j,)) / j
    
    # MC and RQMC
    for r in range(num):
      M = ot.dist(x_mc[r,:j,:], y_mc[r,:j,:], cost)
      M /= M.max()
      mc_ave.append(ot.emd2(a, b, M))
      M = ot.dist(x_rqmc[r,:j,:], y_rqmc[r,:j,:], cost)
      M /= M.max()
      rqmc_ave.append(ot.emd2(a, b, M))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    M = ot.dist(x_qmc[:j,:], y_qmc[:j,:], cost)
    M /= M.max()
    W_qmc.append(np.abs(ot.emd2(a, b, M)))

    print('sample size: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

In [ ]:
 def W_conv_inv(model,n,num,d,theta=None,s=2,cost='euclidean',sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,sobol=sobol)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r_inv(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_gandk_r_inv(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in n:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((j,)) / j 
    b = np.ones((j,)) / j
    
    # MC and RQMC
    for r in range(num):
      M = ot.dist(x_mc[r,:j,:], y_mc[r,:j,:], cost)
      M /= M.max()
      mc_ave.append(ot.emd2(a, b, M))
      M = ot.dist(x_rqmc[r,:j,:], y_rqmc[r,:j,:], cost)
      M /= M.max()
      rqmc_ave.append(ot.emd2(a, b, M))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    M = ot.dist(x_qmc[:j,:], y_qmc[:j,:], cost)
    M /= M.max()
    W_qmc.append(np.abs(ot.emd2(a, b, M)))

    print('sample size: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

Function to calculate Wasserstein distance against $d$ for uniform distribution and Gaussian distribution:

In [ ]:
 def W_conv_d(model,n,num,d,theta=None,s=2,cost='euclidean',sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_d(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_d(n,num,d,s,theta,sobol=sobol)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in d:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((n,)) / n 
    b = np.ones((n,)) / n
    
    # MC and RQMC
    for r in range(num):
      M = ot.dist(x_mc[r,:,:j], y_mc[r,:,:j], cost)
      M /= M.max()
      mc_ave.append(ot.emd2(a, b, M))
      M = ot.dist(x_rqmc[r,:,:j], y_rqmc[r,:,:j], cost)
      M /= M.max()
      rqmc_ave.append(ot.emd2(a, b, M))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    M = ot.dist(x_qmc[:,:j], y_qmc[:,:j], cost)
    M /= M.max()
    W_qmc.append(np.abs(ot.emd2(a, b, M)))

    print('Number of dimensions: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

In [ ]:
 def W_conv_inv_d(model,n,num,d,theta=None,s=2,cost='euclidean',sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,sobol=sobol)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in d:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((n,)) / n 
    b = np.ones((n,)) / n
    
    # MC and RQMC
    for r in range(num):
      M = ot.dist(x_mc[r,:,:j], y_mc[r,:,:j], cost)
      M /= M.max()
      mc_ave.append(ot.emd2(a, b, M))
      M = ot.dist(x_rqmc[r,:,:j], y_rqmc[r,:,:j], cost)
      M /= M.max()
      rqmc_ave.append(ot.emd2(a, b, M))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    M = ot.dist(x_qmc[:,:j], y_qmc[:,:j], cost)
    M /= M.max()
    W_qmc.append(np.abs(ot.emd2(a, b, M)))

    print('Number of dimensions: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

### Convergence of Sinkhorn divergence

Function to calculate Sinkhorn divergence against $n$ for uniform distribution, Gaussian distribution, bivariate beta distribution and g-and-k distribution:

In [ ]:
def sink_conv(model,n,num,d,e,theta=None,s=2, method='sinkhorn',cost='sqeuclidean',sobol=False):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r(n,num,d,s,theta,sobol=sobol)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r(n,num,d,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_gandk_r(n,num,d,theta)

  # calculate Sinkhorn distance for a sequence of n
  sink_qmc = []
  sink_mc = []
  sink_rqmc = []
  sink_min_mc = []
  sink_max_mc = []
  sink_min_rqmc = []
  sink_max_rqmc = []
  for j in n:

    # equal weights
    a = np.ones((j,)) / j  
    b = np.ones((j,)) / j

    #MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      mc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_mc[r,:j,:], y_mc[r,:j,:], e, a, b, cost, method=method))
      rqmc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_rqmc[r,:j,:], y_rqmc[r,:j,:], e, a, b, cost, method=method))

    # QMC
    sink_qmc.append(ot.bregman.empirical_sinkhorn_divergence(x_qmc[:j,:], y_qmc[:j,:], e, a, b, cost, method=method))

    # calculate mean Sinkhorn loss for MC and RQMC
    sink_mc.append(np.mean(np.array(mc_ave)))
    sink_rqmc.append(np.mean(np.array(rqmc_ave)))

    # calculate min and max values for MC and RQMC
    sink_max_mc.append(np.squeeze(np.max(np.array(mc_ave))))
    sink_min_mc.append(np.squeeze(np.min(np.array(mc_ave))))
    sink_max_rqmc.append(np.squeeze(np.max(np.array(rqmc_ave))))
    sink_min_rqmc.append(np.squeeze(np.min(np.array(rqmc_ave))))

    print('sample size: ', j)

  return list([sink_mc,sink_qmc,sink_rqmc,sink_min_mc,sink_max_mc,sink_min_rqmc,sink_max_rqmc])

In [ ]:
def sink_conv_inv(model,n,num,d,e,theta=None,s=2, method='sinkhorn',cost='sqeuclidean',sobol=False):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,sobol=sobol)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_inv(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_inv(n,num,d,theta)
  if model == 'gandk':
    x_mc,x_qmc,x_rqmc = sample_gandk_r_inv(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_gandk_r_inv(n,num,d,theta)

  # calculate Sinkhorn distance for a sequence of n
  sink_qmc = []
  sink_mc = []
  sink_rqmc = []
  sink_min_mc = []
  sink_max_mc = []
  sink_min_rqmc = []
  sink_max_rqmc = []
  for j in n:

    # equal weights
    a = np.ones((j,)) / j  
    b = np.ones((j,)) / j

    #MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      mc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_mc[r,:j,:], y_mc[r,:j,:], e, a, b, cost, method=method))
      rqmc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_rqmc[r,:j,:], y_rqmc[r,:j,:], e, a, b, cost, method=method))

    # QMC
    sink_qmc.append(ot.bregman.empirical_sinkhorn_divergence(x_qmc[:j,:], y_qmc[:j,:], e, a, b, cost, method=method))

    # calculate mean Sinkhorn loss for MC and RQMC
    sink_mc.append(np.mean(np.array(mc_ave)))
    sink_rqmc.append(np.mean(np.array(rqmc_ave)))

    # calculate min and max values for MC and RQMC
    sink_max_mc.append(np.squeeze(np.max(np.array(mc_ave))))
    sink_min_mc.append(np.squeeze(np.min(np.array(mc_ave))))
    sink_max_rqmc.append(np.squeeze(np.max(np.array(rqmc_ave))))
    sink_min_rqmc.append(np.squeeze(np.min(np.array(rqmc_ave))))

    print('sample size: ', j)

  return list([sink_mc,sink_qmc,sink_rqmc,sink_min_mc,sink_max_mc,sink_min_rqmc,sink_max_rqmc])

Function to calculate Sinkhorn loss against $d$ for uniform distribution, Gaussian distribution and g-and-k distribution:

In [ ]:
def sink_conv_d(model,n,num,d,e,theta=None,s=2,eadapt=False,method='sinkhorn',cost='sqeuclidean',sobol=False):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_d(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_d(n,num,d,s,theta,sobol=sobol)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_d(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_d(n,num,d,theta)

  # calculate Sinkhorn distance for a sequence of n
  sink_qmc = []
  sink_mc = []
  sink_rqmc = []
  sink_min_mc = []
  sink_max_mc = []
  sink_min_rqmc = []
  sink_max_rqmc = []
  for j in d:

    # equal weights
    a = np.ones((n,)) / n  
    b = np.ones((n,)) / n

    #MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      if eadapt:
        mc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_mc[r,:,:j], y_mc[r,:,:j], e*j, a, b, cost, method=method))
        rqmc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_rqmc[r,:,:j], y_rqmc[r,:,:j], e*j, a, b, cost, method=method))
      else:
        mc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_mc[r,:,:j], y_mc[r,:,:j], e, a, b, cost, method=method))
        rqmc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_rqmc[r,:,:j], y_rqmc[r,:,:j], e, a, b, cost, method=method))

    # QMC
    if eadapt:
      sink_qmc.append(ot.bregman.empirical_sinkhorn_divergence(x_qmc[:,:j], y_qmc[:,:j], e*j, a, b, cost, method=method))
    else:
      sink_qmc.append(ot.bregman.empirical_sinkhorn_divergence(x_qmc[:,:j], y_qmc[:,:j], e, a, b, cost, method=method))

    # calculate mean Sinkhorn loss for MC and RQMC
    sink_mc.append(np.mean(np.array(mc_ave)))
    sink_rqmc.append(np.mean(np.array(rqmc_ave)))

    # calculate min and max values for MC and RQMC
    sink_max_mc.append(np.squeeze(np.max(np.array(mc_ave))))
    sink_min_mc.append(np.squeeze(np.min(np.array(mc_ave))))
    sink_max_rqmc.append(np.squeeze(np.max(np.array(rqmc_ave))))
    sink_min_rqmc.append(np.squeeze(np.min(np.array(rqmc_ave))))

    print('number of dimensions: ', j)

  return list([sink_mc,sink_qmc,sink_rqmc,sink_min_mc,sink_max_mc,sink_min_rqmc,sink_max_rqmc])

In [ ]:
def sink_conv_inv_d(model,n,num,d,e,theta=None,s=2,eadapt=False, method='sinkhorn',cost='sqeuclidean',sobol=False):

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,sobol=sobol)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_inv_d(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_inv_d(n,num,d,theta)

  # calculate Sinkhorn distance for a sequence of n
  sink_qmc = []
  sink_mc = []
  sink_rqmc = []
  sink_min_mc = []
  sink_max_mc = []
  sink_min_rqmc = []
  sink_max_rqmc = []
  for j in d:

    # equal weights
    a = np.ones((n,)) / n  
    b = np.ones((n,)) / n

    #MC and RQMC
    mc_ave = []
    rqmc_ave = []
    for r in range(num):
      if eadapt:
        mc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_mc[r,:,:j], y_mc[r,:,:j], e*j, a, b, cost, method=method))
        rqmc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_rqmc[r,:,:j], y_rqmc[r,:,:j], e*j, a, b, cost, method=method))
      else:
        mc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_mc[r,:,:j], y_mc[r,:,:j], e, a, b, cost, method=method))
        rqmc_ave.append(ot.bregman.empirical_sinkhorn_divergence(x_rqmc[r,:,:j], y_rqmc[r,:,:j], e, a, b, cost, method=method))

    # QMC
    if eadapt:
      sink_qmc.append(ot.bregman.empirical_sinkhorn_divergence(x_qmc[:,:j], y_qmc[:,:j], e*j, a, b, cost, method=method))
    else:
      sink_qmc.append(ot.bregman.empirical_sinkhorn_divergence(x_qmc[:,:j], y_qmc[:,:j], e, a, b, cost, method=method))

    # calculate mean Sinkhorn loss for MC and RQMC
    sink_mc.append(np.mean(np.array(mc_ave)))
    sink_rqmc.append(np.mean(np.array(rqmc_ave)))

    # calculate min and max values for MC and RQMC
    sink_max_mc.append(np.squeeze(np.max(np.array(mc_ave))))
    sink_min_mc.append(np.squeeze(np.min(np.array(mc_ave))))
    sink_max_rqmc.append(np.squeeze(np.max(np.array(rqmc_ave))))
    sink_min_rqmc.append(np.squeeze(np.min(np.array(rqmc_ave))))

    print('number of dimensions: ', j)

  return list([sink_mc,sink_qmc,sink_rqmc,sink_min_mc,sink_max_mc,sink_min_rqmc,sink_max_rqmc])

### Convergence of sliced Wasserstein distance

Function to calculate sliced Wasserstein distance against $n$ for uniform, Gaussian distribution bivariate beta distribution and g-and-k distribution:

In [ ]:
 def slicedW_conv(model,n,num,d,n_projections,metric,theta=None,s=2,sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r(n,num,d,s,theta,sobol=sobol)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in n:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((j,)) / j 
    b = np.ones((j,)) / j
    
    # MC and RQMC
    for r in range(num):
      mc_ave.append(sliced_wasserstein_distance(x_mc[r,:j,:],y_mc[r,:j,:], metric, a, b, n_projections))
      rqmc_ave.append(sliced_wasserstein_distance(x_rqmc[r,:j,:],y_rqmc[r,:j,:], metric, a, b, n_projections))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    W_qmc.append(np.abs(sliced_wasserstein_distance(x_qmc[:j,:],y_qmc[:j,:], metric, a, b, n_projections)))

    print('sample size: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

In [ ]:
 def slicedW_conv_inv(model,n,num,d,n_projections,metric,theta=None,s=2,sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv(n,num,d,s,theta,sobol=sobol)
  if model=='bibeta':
    x_mc,x_qmc,x_rqmc = sample_bibeta_r(n,num,theta)
    y_mc,y_qmc,y_rqmc = sample_bibeta_r(n,num,theta)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_inv(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_inv(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in n:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((j,)) / j 
    b = np.ones((j,)) / j
    
    # MC and RQMC
    for r in range(num):
      mc_ave.append(sliced_wasserstein_distance(x_mc[r,:j,:],y_mc[r,:j,:], metric, a, b, n_projections))
      rqmc_ave.append(sliced_wasserstein_distance(x_rqmc[r,:j,:],y_rqmc[r,:j,:], metric, a, b, n_projections))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    W_qmc.append(np.abs(sliced_wasserstein_distance(x_qmc[:j,:],y_qmc[:j,:], metric, a, b, n_projections)))

    print('sample size: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

Function to calculate sliced Wasserstein distance against $d$ for uniform, Gaussian distribution and g-and-k distribution:

In [ ]:
 def slicedW_conv_d(model,n,num,d,n_projections,metric,theta=None,s=2,sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_d(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_d(n,num,d,s,theta,sobol=sobol)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_d(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_d(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in d:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((n,)) / n 
    b = np.ones((n,)) / n
    
    # MC and RQMC
    for r in range(num):
      mc_ave.append(sliced_wasserstein_distance(x_mc[r,:,:j],y_mc[r,:,:j], metric, a, b, n_projections))
      rqmc_ave.append(sliced_wasserstein_distance(x_rqmc[r,:,:j],y_rqmc[r,:,:j], metric, a, b, n_projections))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    W_qmc.append(np.abs(sliced_wasserstein_distance(x_qmc[:,:j],y_qmc[:,:j], metric, a, b, n_projections)))

    print('Number of dimensions: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])

In [ ]:
 def slicedW_conv_inv_d(model,n,num,d,n_projections,metric,theta=None,s=2,sobol=False): 

  # generate samples
  if model=='unif':
    x_mc,x_qmc,x_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_unif_r_d(n,num,d,sobol=sobol)
  if model=='gaussian':
    x_mc,x_qmc,x_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,sobol=sobol)
    y_mc,y_qmc,y_rqmc = sample_gaussian_r_inv_d(n,num,d,s,theta,sobol=sobol)
  if model == 'mvgandk':
    x_mc,x_qmc,x_rqmc = sample_mvgandk_r_inv_d(n,num,d,theta)
    y_mc,y_qmc,y_rqmc = sample_mvgandk_r_inv_d(n,num,d,theta)

  # calculate Wasserstein distance for a sequence of n
  W_mc = []
  W_qmc = []
  W_rqmc = []
  W_min_mc = []
  W_max_mc = []
  W_min_rqmc = []
  W_max_rqmc = []
  for j in d:

    mc_ave = []
    rqmc_ave = []

    # equal weights
    a = np.ones((n,)) / n 
    b = np.ones((n,)) / n
    
    # MC and RQMC
    for r in range(num):
      mc_ave.append(sliced_wasserstein_distance(x_mc[r,:,:j],y_mc[r,:,:j], metric, a, b, n_projections))
      rqmc_ave.append(sliced_wasserstein_distance(x_rqmc[r,:,:j],y_rqmc[r,:,:j], metric, a, b, n_projections))
    W_mc.append(np.mean(np.abs(np.array(mc_ave))))
    W_rqmc.append(np.mean(np.abs(np.array(rqmc_ave))))
    
    # append min and max values for MC and RQMC
    W_min_mc.append(np.min(np.abs(np.array(mc_ave))))
    W_max_mc.append(np.max(np.abs(np.array(mc_ave))))
    W_min_rqmc.append(np.min(np.abs(np.array(rqmc_ave))))
    W_max_rqmc.append(np.max(np.abs(np.array(rqmc_ave)))) 

    # QMC
    W_qmc.append(np.abs(sliced_wasserstein_distance(x_qmc[:,:j],y_qmc[:,:j], metric, a, b, n_projections)))

    print('Number of dimensions: ', j)

  return list([W_mc,W_qmc,W_rqmc,W_min_mc,W_max_mc,W_min_rqmc,W_max_rqmc])